In [ ]:
import os
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop, Adam, SGD, Adamax

print('direktori saat ini ' + os.getcwd())
print('versi tensorflow ' + tf.__version__)

In [3]:
import os
data_dir = './data'
print(os.listdir(data_dir))
test_dir = os.path.join(data_dir, 'test')
train_dir = os.path.join(data_dir, 'train')
print('Direktori train dataset: ')
print(os.listdir(train_dir))
print('Direktori test dataset: ')
print(os.listdir(test_dir))

['test', 'train']
Direktori train dataset: 
['bench_press', 'bicycle', 'leg_press', 'pec_deck', 'rowing', 'treadmill']
Direktori test dataset: 
['bench_press', 'bicycle', 'leg_press', 'pec_deck', 'rowing', 'treadmill']


In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=16,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    shear_range=0.2,
    zoom_range=0.1,
    fill_mode='nearest'
)
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_dataset = train_datagen.flow_from_directory(
    train_dir,
    target_size=(256, 256),
    class_mode='categorical',
    batch_size=32
)

test_dataset = test_datagen.flow_from_directory(
    test_dir,
    target_size=(256, 256),
    class_mode='categorical',
    batch_size=32
)

In [ ]:
train_dataset.class_indices

In [ ]:
train_dataset

In [ ]:
train_dataset.batch_size

In [ ]:
labels = (train_dataset.class_indices)
labels = dict((v,k) for k,v in labels.items())

In [ ]:
print(labels)

In [ ]:
jml_class = len(labels)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', padding='same', input_shape=(256, 256, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(jml_class, activation='softmax')
    
])
model.summary()

In [ ]:
from tensorflow.keras.optimizers import RMSprop, Adam, SGD, Adamax
model.compile(optimizer = Adam(learning_rate=1e-3), loss=tf.losses.CategoricalCrossentropy(), metrics=['accuracy'])

In [ ]:
history = model.fit(train_dataset, epochs=100, validation_data=test_dataset)

In [ ]:
import matplotlib.pyplot as plt
#-----------------------------------------------------------
# Retrieve a list of list results on training and test data
# sets for each training epoch
#-----------------------------------------------------------
acc      = history.history['accuracy']
val_acc  = history.history['val_accuracy']
loss     = history.history['loss']
val_loss = history.history['val_loss']

epochs   = range(len(acc)) # Get number of epochs

#------------------------------------------------
# Plot training and validation accuracy per epoch
#------------------------------------------------
plt.plot  ( epochs, acc )
plt.plot  ( epochs, val_acc )
plt.title ('Training and validation accuracy')
plt.figure()

#------------------------------------------------
# Plot training and validation loss per epoch
#------------------------------------------------
plt.plot  ( epochs, loss )
plt.plot  ( epochs, val_loss )
plt.title ('Training and validation loss'   )

In [ ]:
export_dir = 'saved_model'
tf.saved_model.save(model, export_dir)

In [ ]:
optimization = tf.lite.Optimize.OPTIMIZE_FOR_SIZE

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

In [ ]:
import pathlib
tflite_model_file = pathlib.Path('./model.tflite')
tflite_model_file.write_bytes(tflite_model)

In [ ]:
img_url = "https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fsc01.alicdn.com%2Fkf%2FHTB1L21sXQH0gK0jSZFNq6xMqXXai%2F230837379%2FHTB1L21sXQH0gK0jSZFNq6xMqXXai.jpg&f=1&nofb=1&ipt=7bf76cad8fbb85673b335ce0f0c5539272279b5159930649aa12f077cb5996e6&ipo=images"
img_path = tf.keras.utils.get_file('leg-press-machine', origin=img_url)

img = tf.keras.utils.load_img(
    img_path, target_size=(256, 256)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(labels[np.argmax(score)], 100 * np.max(score))
)